In [0]:
# Create widgets for Gold layer
dbutils.widgets.text("source_table", "default.silver_events")

source_table = dbutils.widgets.get("source_table")

print("Running Gold layer")
print("Source Table:", source_table)

In [0]:
from pyspark.sql import functions as F

gold_df = (
    spark.read.table(source_table)
    .groupBy("product_id")
    .agg(
        F.count(F.when(F.col("event_type") == "view", True)).alias("views"),
        F.count(F.when(F.col("event_type") == "purchase", True)).alias("purchases"),
        F.sum(F.when(F.col("event_type") == "purchase", F.col("price"))).alias("revenue")
    )

    .withColumn(
        "conversion_rate",
        F.when(F.col("views") > 0, (F.col("purchases") / F.col("views")) * 100)
         .otherwise(0)
    )
)

gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.gold_product_performance")

print("Gold layer completed successfully")

In [0]:
spark.read.table("default.gold_product_performance").show(5)